# Pràctica 3

In [8]:
#%pip install python-crfsuite

In [9]:
import nltk
import pycrfsuite
from nltk.corpus import conll2002
from nltk.tag import CRFTagger
from sklearn.metrics import accuracy_score

In [10]:
nltk.download('punkt') # Tokenitzador
nltk.download('averaged_perceptron_tagger') # Etiquetador POS
nltk.download('maxent_ne_chunker') # Etiquetador Entitats Anomenades
nltk.download('words')
nltk.download('treebank')
nltk.download('conll2002')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-d

True

In [11]:
train_esp = conll2002.iob_sents('esp.train') # Train, ned.train => Neerlandès
testa_esp = conll2002.iob_sents('esp.testa') # Dev
testb_esp = conll2002.iob_sents('esp.testb') # Test

In [12]:
train_ned = conll2002.iob_sents('ned.train') # Train, ned.train => Neerlandès
testa_ned = conll2002.iob_sents('ned.testa') # Dev
testb_ned = conll2002.iob_sents('ned.testb') # Test

## Predicció amb BIO

### Espanyol

In [13]:
model_tagger = CRFTagger()

In [14]:
# Entrenem el model per predir els POS que corresponen a cada token

train_esp_pos_tag = []
for sentence in train_esp:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem2))
    train_esp_pos_tag.append(frases)
    
model_tagger.train(train_esp_pos_tag, 'model_POS.crf.tagger')


# Fem prediccions i mirem l'accuracy

testa_esp_pre_tag = []
for sentence in testa_esp:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append(elem1)
    testa_esp_pre_tag.append(frases)
    
predicted = model_tagger.tag_sents(testa_esp_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in testa_esp for elem in sentence]

print(accuracy_score(predictions, real_label))

0.9447121289420479


In [15]:
from nltk.tag import CRFTagger
import unicodedata
import re


class FeatureExtractor2:
    def __init__(self, use_basic_features=False, use_prefix_suffix_features=False, use_context_features=False, pattern = r'\d+'):
        self.use_basic_features = use_basic_features
        self.use_prefix_suffix_features = use_prefix_suffix_features
        self.use_context_features = use_context_features
        self._pattern = pattern
        self.feature_list = []

    def _extract_basic_features(self, token):
        # Capitalization
        if token[0].isupper():
            self.feature_list.append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token) is not None:
            self.feature_list.append("HAS_NUM")

        # Punctuation
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token):
            self.feature_list.append("PUNCTUATION")

    def _extract_prefix_suffix_features(self, token):
        # preffix up to length 3
        if len(token) > 1:
            self.feature_list.append("PRE_" + token[:1])
        if len(token) > 2:
            self.feature_list.append("PRE_" + token[:2])
        if len(token) > 3:
            self.feature_list.append("PRE_" + token[:3])

        # Suffix up to length 3
        if len(token) > 1:
            self.feature_list.append("SUF_" + token[-1:])
        if len(token) > 2:
            self.feature_list.append("SUF_" + token[-2:])
        if len(token) > 3:
            self.feature_list.append("SUF_" + token[-3:])

    def _extract_context_features(self, token, tokens, idx):
        # POS_tags
        POS = model_tagger.tag(tokens)
            
        # Paraules prèvies amb POS
        if idx > 0:
            self.feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
        if idx > 1:
            self.feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
            
        # Paraules posteriors amb POS
        if idx < (len(tokens)-1):
            self.feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
        if idx < (len(tokens)-2):
            self.feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

        self.feature_list.append("WORD_" + token)
    
    def _get_features(self, tokens, idx):
        token = tokens[idx]
        feature_list = []
        
        if self.use_basic_features:
            self._extract_basic_features(token)
                    
        if self.use_prefix_suffix_features:
            self._extract_prefix_suffix_features(token)
        
        if self.use_context_features:
            self._extract_context_features(token, tokens, idx)
            
        
        return feature_list

In [16]:
feature_extractor = FeatureExtractor2(use_basic_features=False, use_prefix_suffix_features=False, use_context_features=False, pattern = r'\d+')

train_esp_BIO_tag = []
for sentence in train_esp:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem3))
    train_esp_BIO_tag.append(frases)
    
model_BIO = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO.train(train_esp_BIO_tag, 'model_BIO.crf.tagger')

<span> Features que es tenen en compte:
<ul>
    <li>Paraula actual</li>
    <li>Si comença en majúscula</li>
    <li>Si té signe de puntuació</li>
    <li>Si té números</li>
    <li>Prefixos fins a longitud 3</li>
    <li>Sufixos fins a longitud 3</li>
    <li>Paraules prèvies i posteriors amb POS</li>
    <li>POS-tags</li>
    <li>Longitud de la paraula</li>
</ul>
</span>

In [17]:
from nltk.tag import CRFTagger
import unicodedata
import re

class FeatureExtractor:
    def __init__(self, pattern):
        self._pattern = pattern

    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags
            - longitud

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
        token = tokens[idx]

        feature_list = []

        if not token:
            return feature_list

        # Capitalization
        if token[0].isupper():
            feature_list.append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token) is not None:
            feature_list.append("HAS_NUM")

        # Punctuation
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token):
            feature_list.append("PUNCTUATION")
            
        # preffix up to length 3
        if len(token) > 1:
            feature_list.append("PRE_" + token[:1])
        if len(token) > 2:
            feature_list.append("PRE_" + token[:2])
        if len(token) > 3:
            feature_list.append("PRE_" + token[:3])

        # Suffix up to length 3
        if len(token) > 1:
            feature_list.append("SUF_" + token[-1:])
        if len(token) > 2:
            feature_list.append("SUF_" + token[-2:])
        if len(token) > 3:
            feature_list.append("SUF_" + token[-3:])
        
        # POS_tags
        POS = model_tagger.tag(tokens)
            
        # Paraules prèvies amb POS
        if idx > 0:
            feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
        if idx > 1:
            feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
            
        # Paraules posteriors amb POS
        if idx < (len(tokens)-1):
            feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
        if idx < (len(tokens)-2):
            feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

        feature_list.append("WORD_" + token)

        return feature_list

# Crear una instancia de FeatureExtractor
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_esp_BIO_tag1 = []
for sentence in train_esp:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem3))
    train_esp_BIO_tag1.append(frases)
    
model_BIO = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO.train(train_esp_BIO_tag1, 'model_BIO.crf.tagger')

#### Utilitzem aquest accuracy per ajustar els features

In [18]:
testa_esp_BIO_tag = []
for sentence in testa_esp:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem3))
    testa_esp_BIO_tag.append(frases)

predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)

predictions_BIO = [elem[1] for sentence in predicted_BIO for elem in sentence]
real_label_BIO = [elem[1] for sentence in testa_esp_BIO_tag for elem in sentence]

print(accuracy_score(predictions_BIO, real_label_BIO))

0.9580711599871511


#### Reconeixem entitats i avaluem la detecció d'entitats

In [19]:
entitats_reals_testa_esp = []

for sentence in testa_esp_BIO_tag:
    ent = []
    name = None
    prev_tag = None  # Almacenar la etiqueta del token anterior
    
    for token in sentence:
        word, tag = token
        
        if tag.startswith('B-'):
            # Si hay una entidad anterior, la agregamos a la lista de entidades
            if ent:
                entitats_reals_testa_esp.append((tuple(ent), name))
            # Creamos una nueva entidad con la palabra actual
            ent = [word]
            # Obtenemos el tipo de entidad
            name = tag.split('-')[1]
            prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag.startswith('I-'):
            # Solo agregamos la palabra actual si el token anterior tiene etiqueta I- o B-
            if prev_tag:
                ent.append(word)
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag == 'O' and ent:
            # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
            entitats_reals_testa_esp.append((tuple(ent), name))
            # Reiniciamos la lista de la entidad actual
            ent = []
            prev_tag = None  # Reiniciamos la etiqueta del token anterior

    # Agregamos la última entidad si la hay
    if ent:
        entitats_reals_testa_esp.append((tuple(ent), name))     

In [20]:
predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)

entitats_predites_testa_esp = []

for sentence in predicted_BIO:
    ent = []
    name = None
    prev_tag = None  # Almacenar la etiqueta del token anterior
    
    for token in sentence:
        word, tag = token
        
        if tag.startswith('B-'):
            # Si hay una entidad anterior, la agregamos a la lista de entidades
            if ent:
                entitats_predites_testa_esp.append((tuple(ent), name))
            # Creamos una nueva entidad con la palabra actual
            ent = [word]
            # Obtenemos el tipo de entidad
            name = tag.split('-')[1]
            prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag.startswith('I-'):
            # Solo agregamos la palabra actual si el token anterior tiene etiqueta I- o B-
            if prev_tag:
                ent.append(word)
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag == 'O' and ent:
            # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
            entitats_predites_testa_esp.append((tuple(ent), name))
            # Reiniciamos la lista de la entidad actual
            ent = []
            prev_tag = None  # Reiniciamos la etiqueta del token anterior

    # Agregamos la última entidad si la hay
    if ent:
        entitats_predites_testa_esp.append((tuple(ent), name))

In [21]:
### AVALUEM ###
encerts = 0
for entitat in entitats_predites_testa_esp:
    if entitat in entitats_reals_testa_esp:
        encerts += 1

total_entitats_reals = len(entitats_reals_testa_esp)
total_entitats_predites = len(entitats_predites_testa_esp)

recall = encerts / total_entitats_reals

if total_entitats_predites != 0:
    precision = encerts / total_entitats_predites
    f_score = (2 * precision * recall) / (precision + recall)
else:
    precision = 0
    f_score = 0

print("Recall:", recall)
print("Precision:", precision)
print("F-score:", f_score)


Recall: 0.7747644219719605
Precision: 0.8156302927655457
F-score: 0.7946723243752947


### Neerlandès

In [22]:
model_tagger = CRFTagger()

In [23]:
# Entrenem el model per predir els POS que corresponen a cada token

train_ned_pos_tag = []
for sentence in train_ned:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem2))
    train_ned_pos_tag.append(frases)
    
model_tagger.train(train_ned_pos_tag, 'model_POS.crf.tagger')


# Fem prediccions i mirem l'accuracy

testa_ned_pre_tag = []
for sentence in testa_ned:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append(elem1)
    testa_ned_pre_tag.append(frases)
    
predicted = model_tagger.tag_sents(testa_ned_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in testa_ned for elem in sentence]

print(accuracy_score(predictions, real_label))

0.940191577997718


In [24]:
from nltk.tag import CRFTagger
import unicodedata
import re

class FeatureExtractor:
    def __init__(self, pattern):
        self._pattern = pattern

    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags
            - longitud

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
        token = tokens[idx]

        feature_list = []

        if not token:
            return feature_list

        # Capitalization
        if token[0].isupper():
            feature_list.append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token) is not None:
            feature_list.append("HAS_NUM")

        # Punctuation
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token):
            feature_list.append("PUNCTUATION")
            
        # preffix up to length 3
        if len(token) > 1:
            feature_list.append("PRE_" + token[:1])
        if len(token) > 2:
            feature_list.append("PRE_" + token[:2])
        if len(token) > 3:
            feature_list.append("PRE_" + token[:3])

        # Suffix up to length 3
        if len(token) > 1:
            feature_list.append("SUF_" + token[-1:])
        if len(token) > 2:
            feature_list.append("SUF_" + token[-2:])
        if len(token) > 3:
            feature_list.append("SUF_" + token[-3:])
        
        # POS_tags
        POS = model_tagger.tag(tokens)
            
        # Paraules prèvies amb POS
        if idx > 0:
            feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
        if idx > 1:
            feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
            
        # Paraules posteriors amb POS
        if idx < (len(tokens)-1):
            feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
        if idx < (len(tokens)-2):
            feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

        feature_list.append("WORD_" + token)

        return feature_list

# Crear una instancia de FeatureExtractor
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_ned_BIO_tag = []
for sentence in train_ned:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem3))
    train_ned_BIO_tag.append(frases)
    
model_BIO = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO.train(train_ned_BIO_tag, 'model_BIO.crf.tagger')

In [25]:
testa_ned_BIO_tag = []
for sentence in testa_ned:
    frases = []
    for elem1, elem2, elem3 in sentence:
        frases.append((elem1, elem3))
    testa_ned_BIO_tag.append(frases)

predicted_BIO = model_BIO.tag_sents(testa_ned_pre_tag)

predictions_BIO = [elem[1] for sentence in predicted_BIO for elem in sentence]
real_label_BIO = [elem[1] for sentence in testa_ned_BIO_tag for elem in sentence]

print(accuracy_score(predictions_BIO, real_label_BIO))

0.9683975906811367


In [26]:
entitats_reals_testa_ned = []

for sentence in testa_ned_BIO_tag:
    ent = []
    name = None
    prev_tag = None  # Almacenar la etiqueta del token anterior
    
    for token in sentence:
        word, tag = token
        
        if tag.startswith('B-'):
            # Si hay una entidad anterior, la agregamos a la lista de entidades
            if ent:
                entitats_reals_testa_ned.append((tuple(ent), name))
            # Creamos una nueva entidad con la palabra actual
            ent = [word]
            # Obtenemos el tipo de entidad
            name = tag.split('-')[1]
            prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag.startswith('I-'):
            # Solo agregamos la palabra actual si el token anterior tiene etiqueta I- o B-
            if prev_tag:
                ent.append(word)
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag == 'O' and ent:
            # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
            entitats_reals_testa_ned.append((tuple(ent), name))
            # Reiniciamos la lista de la entidad actual
            ent = []
            prev_tag = None  # Reiniciamos la etiqueta del token anterior

    # Agregamos la última entidad si la hay
    if ent:
        entitats_reals_testa_ned.append((tuple(ent), name))     

In [27]:
predicted_BIO = model_BIO.tag_sents(testa_ned_pre_tag)

entitats_predites_testa_ned = []

for sentence in predicted_BIO:
    ent = []
    name = None
    prev_tag = None  # Almacenar la etiqueta del token anterior
    
    for token in sentence:
        word, tag = token
        
        if tag.startswith('B-'):
            # Si hay una entidad anterior, la agregamos a la lista de entidades
            if ent:
                entitats_predites_testa_ned.append((tuple(ent), name))
            # Creamos una nueva entidad con la palabra actual
            ent = [word]
            # Obtenemos el tipo de entidad
            name = tag.split('-')[1]
            prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag.startswith('I-'):
            # Solo agregamos la palabra actual si el token anterior tiene etiqueta I- o B-
            if prev_tag:
                ent.append(word)
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
        elif tag == 'O' and ent:
            # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
            entitats_predites_testa_ned.append((tuple(ent), name))
            # Reiniciamos la lista de la entidad actual
            ent = []
            prev_tag = None  # Reiniciamos la etiqueta del token anterior

    # Agregamos la última entidad si la hay
    if ent:
        entitats_predites_testa_ned.append((tuple(ent), name))

In [28]:
### AVALUEM ###
encerts = 0
for entitat in entitats_predites_testa_ned:
    if entitat in entitats_reals_testa_ned:
        encerts += 1

total_entitats_reals = len(entitats_reals_testa_ned)
total_entitats_predites = len(entitats_predites_testa_ned)

recall = encerts / total_entitats_reals

if total_entitats_predites != 0:
    precision = encerts / total_entitats_predites
    f_score = (2 * precision * recall) / (precision + recall)
else:
    precision = 0
    f_score = 0

print("Recall:", recall)
print("Precision:", precision)
print("F-score:", f_score)


Recall: 0.6788990825688074
Precision: 0.7477894736842106
F-score: 0.7116810258465238


## Predicció amb IO

In [29]:
def convert_to_io(train_data_bio):
    train_data_io = []
    for sentence in train_data_bio:
        io_tags = []
        for word, pos_tag, bio_tag in sentence:
            if bio_tag == 'O':
                io_tags.append('O')
            elif bio_tag.startswith('B-'):
                io_tags.append('I' + bio_tag[1:])
            else:
                io_tags.append(bio_tag)
                
        train_data_io.append(list(zip([word for word, pos_tag, bio_tag in sentence], io_tags)))
    return train_data_io


Espanyol

In [30]:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_esp_pre_IO = convert_to_io(train_esp)

# Entrenar el modelo CRFTagger con el esquema IO
model_IO_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_esp.train(train_esp_pre_IO, 'model_io.crf.tagger')

In [31]:
# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format IO
dades_prova = convert_to_io(testa_esp)
prediccions = model_IO_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")


Total d'entitats: 52923
Entitats predites correctament: 50064
Precisió d'entitats: 94.59781191542429 %


In [32]:
from itertools import product

# Llista de totes les combinacions possibles de valors booleans pels paràmetres
param_combinations = list(product([True, False], repeat=3))

for params in param_combinations:
    # Desempaquetem els valors booleans pels paràmetres
    use_basic_features, use_prefix_suffix_features, use_context_features = params
    
    print(f"Prova amb use_basic_features={use_basic_features}, "
          f"use_prefix_suffix_features={use_prefix_suffix_features}, "
          f"use_context_features={use_context_features}:")
    
    # Creem l'extractor de característiques amb els paràmetres corresponents
    feature_extractor = FeatureExtractor2(use_basic_features=use_basic_features, 
                                           use_prefix_suffix_features=use_prefix_suffix_features, 
                                           use_context_features=use_context_features, 
                                           pattern=r'\d+')

    train_esp_pre_IO = convert_to_io(train_esp)

    # Entrenem el model CRFTagger amb l'esquema IO
    model_IO_esp = CRFTagger(feature_func=feature_extractor._get_features)
    model_IO_esp.train(train_esp_pre_IO, 'model_io.crf.tagger')

    # Inicialitzem els comptadors
    total_entitats = 0
    entitats_correctes = 0

    # Convertim les dades de prova a format IO
    dades_prova = convert_to_io(testa_esp)
    prediccions = model_IO_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

    # Iterem sobre cada mostra en el conjunt de dades de prova
    for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
        for paraula_certes, etiqueta_certes in frase_certes:
            for paraula_pred, etiqueta_pred in frase_prediccions:
                if paraula_certes == paraula_pred:
                    total_entitats += 1
                    if etiqueta_certes == etiqueta_pred:
                        entitats_correctes += 1
                    break

    # Calculem el percentatge d'entitats predites correctament
    precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

    print("Total d'entitats:", total_entitats)
    print("Entitats predites correctament:", entitats_correctes)
    print("Precisió d'entitats:", precisio_entitats, "%")
    print("-" * 50)


Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=False:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=True:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=False:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------


In [ ]:
feature_extractor = FeatureExtractor2(use_basic_features = False, use_prefix_suffix_features = False, use_context_features = False, pattern = r'\d+')

train_esp_pre_IO = convert_to_io(train_esp)

# Entrenar el modelo CRFTagger con el esquema IO
model_IO_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_esp.train(train_esp_pre_IO, 'model_io.crf.tagger')

# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format IO
dades_prova = convert_to_io(testb_esp)
prediccions = model_IO_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")


IO NED

In [33]:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_ned_pre_IO = convert_to_io(train_ned)

# Entrenar el modelo CRFTagger con el esquema IO
model_IO_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_ned.train(train_ned_pre_IO, 'model_io.crf.tagger')

In [34]:
# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format IO
dades_prova = convert_to_io(testa_ned)
prediccions = model_IO_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

Total d'entitats: 37687
Entitats predites correctament: 36539
Precisió d'entitats: 96.95385676758565 %


In [35]:
from itertools import product

# Llista de totes les combinacions possibles de valors booleans pels paràmetres
param_combinations = list(product([True, False], repeat=3))

for params in param_combinations:
    # Desempaquetem els valors booleans pels paràmetres
    use_basic_features, use_prefix_suffix_features, use_context_features = params
    
    print(f"Prova amb use_basic_features={use_basic_features}, "
          f"use_prefix_suffix_features={use_prefix_suffix_features}, "
          f"use_context_features={use_context_features}:")
    
    # Creem l'extractor de característiques amb els paràmetres corresponents
    feature_extractor = FeatureExtractor2(use_basic_features=use_basic_features, 
                                           use_prefix_suffix_features=use_prefix_suffix_features, 
                                           use_context_features=use_context_features, 
                                           pattern=r'\d+')

    train_ned_pre_IO = convert_to_io(train_ned)

    # Entrenem el model CRFTagger amb l'esquema IO
    model_IO_esp = CRFTagger(feature_func=feature_extractor._get_features)
    model_IO_esp.train(train_ned_pre_IO, 'model_io.crf.tagger')

    # Inicialitzem els comptadors
    total_entitats = 0
    entitats_correctes = 0

    # Convertim les dades de prova a format IO
    dades_prova = convert_to_io(testa_ned)
    prediccions = model_IO_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

    # Iterem sobre cada mostra en el conjunt de dades de prova
    for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
        for paraula_certes, etiqueta_certes in frase_certes:
            for paraula_pred, etiqueta_pred in frase_prediccions:
                if paraula_certes == paraula_pred:
                    total_entitats += 1
                    if etiqueta_certes == etiqueta_pred:
                        entitats_correctes += 1
                    break

    # Calculem el percentatge d'entitats predites correctament
    precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

    print("Total d'entitats:", total_entitats)
    print("Entitats predites correctament:", entitats_correctes)
    print("Precisió d'entitats:", precisio_entitats, "%")
    print("-" * 50)


Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=False:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=True:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=False:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------


In [ ]:
feature_extractor = FeatureExtractor2(use_basic_features = False, use_prefix_suffix_features = False, use_context_features = False, pattern = r'\d+')

train_ned_pre_IO = convert_to_io(train_ned)

# Entrenar el modelo CRFTagger con el esquema IO
model_IO_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_ned.train(train_ned_pre_IO, 'model_io.crf.tagger')

# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format IO
dades_prova = convert_to_io(testb_ned)
prediccions = model_IO_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

BIOW

In [36]:
def convert_to_biow(train_data_bio):
    train_data_biow = []
    for sentence in train_data_bio:
        biow_tags = []
        for word, pos_tag, bio_tag in sentence:
            if bio_tag == 'O':
                biow_tags.append('O')
            else:
                biow_tags.append(bio_tag + 'W')  # Añadir 'W' a todas las etiquetas
            
        train_data_biow.append(list(zip([word for word, pos_tag, bio_tag in sentence], biow_tags)))
    return train_data_biow

esp

In [37]:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_esp_pre_BIOW = convert_to_biow(train_esp)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOW_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOW_esp.train(train_esp_pre_BIOW, 'model_biow.crf.tagger')

In [38]:
# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_biow(testa_esp)
prediccions = model_BIOW_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")


Total d'entitats: 52923
Entitats predites correctament: 50166
Precisió d'entitats: 94.79054475369877 %


In [39]:
from itertools import product

# Llista de totes les combinacions possibles de valors booleans pels paràmetres
param_combinations = list(product([True, False], repeat=3))

for params in param_combinations:
    # Desempaquetem els valors booleans pels paràmetres
    use_basic_features, use_prefix_suffix_features, use_context_features = params
    
    print(f"Prova amb use_basic_features={use_basic_features}, "
          f"use_prefix_suffix_features={use_prefix_suffix_features}, "
          f"use_context_features={use_context_features}:")
    
    # Creem l'extractor de característiques amb els paràmetres corresponents
    feature_extractor = FeatureExtractor2(use_basic_features=use_basic_features, 
                                           use_prefix_suffix_features=use_prefix_suffix_features, 
                                           use_context_features=use_context_features, 
                                           pattern=r'\d+')

    train_esp_pre_BIOW = convert_to_biow(train_esp)

    # Entrenem el model CRFTagger amb l'esquema BIOW
    model_BIOW_esp = CRFTagger(feature_func=feature_extractor._get_features)
    model_BIOW_esp.train(train_esp_pre_IO, 'model_biow.crf.tagger')

    # Inicialitzem els comptadors
    total_entitats = 0
    entitats_correctes = 0

    # Convertim les dades de prova a format BIOW
    dades_prova = convert_to_biow(testa_esp)
    prediccions = model_BIOW_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

    # Iterem sobre cada mostra en el conjunt de dades de prova
    for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
        for paraula_certes, etiqueta_certes in frase_certes:
            for paraula_pred, etiqueta_pred in frase_prediccions:
                if paraula_certes == paraula_pred:
                    total_entitats += 1
                    if etiqueta_certes == etiqueta_pred:
                        entitats_correctes += 1
                    break

    # Calculem el percentatge d'entitats predites correctament
    precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

    print("Total d'entitats:", total_entitats)
    print("Entitats predites correctament:", entitats_correctes)
    print("Precisió d'entitats:", precisio_entitats, "%")
    print("-" * 50)

Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=False:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=True:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=False:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------


In [ ]:
feature_extractor = FeatureExtractor2(use_basic_features = False, use_prefix_suffix_features = False, use_context_features = False, pattern = r'\d+')

train_esp_pre_BIOW = convert_to_biow(train_esp)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOW_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOW_esp.train(train_esp_pre_BIOW, 'model_biow.crf.tagger')

# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_biow(testb_esp)
prediccions = model_BIOW_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")


ned

In [40]:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_ned_pre_BIOW = convert_to_biow(train_ned)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOW_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOW_ned.train(train_ned_pre_BIOW, 'model_biow.crf.tagger')

In [41]:
# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_biow(testa_ned)
prediccions = model_BIOW_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

Total d'entitats: 37687
Entitats predites correctament: 36486
Precisió d'entitats: 96.81322471939926 %


In [42]:
from itertools import product

# Llista de totes les combinacions possibles de valors booleans pels paràmetres
param_combinations = list(product([True, False], repeat=3))

for params in param_combinations:
    # Desempaquetem els valors booleans pels paràmetres
    use_basic_features, use_prefix_suffix_features, use_context_features = params
    
    print(f"Prova amb use_basic_features={use_basic_features}, "
          f"use_prefix_suffix_features={use_prefix_suffix_features}, "
          f"use_context_features={use_context_features}:")
    
    # Creem l'extractor de característiques amb els paràmetres corresponents
    feature_extractor = FeatureExtractor2(use_basic_features=use_basic_features, 
                                           use_prefix_suffix_features=use_prefix_suffix_features, 
                                           use_context_features=use_context_features, 
                                           pattern=r'\d+')

    train_ned_pre_BIOW = convert_to_biow(train_ned)

    # Entrenem el model CRFTagger amb l'esquema IO
    model_BIOW_ned = CRFTagger(feature_func=feature_extractor._get_features)
    model_BIOW_ned.train(train_ned_pre_IO, 'model_biow.crf.tagger')

    # Inicialitzem els comptadors
    total_entitats = 0
    entitats_correctes = 0

    # Convertim les dades de prova a format BIOW
    dades_prova = convert_to_biow(testa_ned)
    prediccions = model_BIOW_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

    # Iterem sobre cada mostra en el conjunt de dades de prova
    for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
        for paraula_certes, etiqueta_certes in frase_certes:
            for paraula_pred, etiqueta_pred in frase_prediccions:
                if paraula_certes == paraula_pred:
                    total_entitats += 1
                    if etiqueta_certes == etiqueta_pred:
                        entitats_correctes += 1
                    break

    # Calculem el percentatge d'entitats predites correctament
    precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

    print("Total d'entitats:", total_entitats)
    print("Entitats predites correctament:", entitats_correctes)
    print("Precisió d'entitats:", precisio_entitats, "%")
    print("-" * 50)


Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=False:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=True:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=False:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------


In [ ]:
feature_extractor = FeatureExtractor2(use_basic_features = False, use_prefix_suffix_features = False, use_context_features = False, pattern = r'\d+')

train_ned_pre_BIOW = convert_to_biow(train_ned)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOW_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOW_ned.train(train_ned_pre_BIOW, 'model_biow.crf.tagger')

# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_biow(testb_ned)
prediccions = model_BIOW_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

BIOES

In [43]:
def convert_to_bioes(train_data_bio):
    train_data_bioes = []
    for sentence in train_data_bio:
        bioes_tags = []
        for i, (word, pos_tag, bio_tag) in enumerate(sentence):
            if bio_tag == 'O':
                bioes_tags.append('O')
            elif bio_tag.startswith('B-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('S' + bio_tag[1:])  # Single
                else:
                    bioes_tags.append('B' + bio_tag[1:])  # Begin
            elif bio_tag.startswith('I-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('E' + bio_tag[1:])  # End
                else:
                    bioes_tags.append('I' + bio_tag[1:])  # Inside
            else:
                raise ValueError("Etiqueta BIO incorrecta: {}".format(bio_tag))
                
        train_data_bioes.append(list(zip([word for word, pos_tag, bio_tag in sentence], bioes_tags)))
    return train_data_bioes

esp

In [44]:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_esp_pre_BIOES = convert_to_bioes(train_esp)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOES_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_esp.train(train_esp_pre_BIOW, 'model_bioes.crf.tagger')

In [45]:
# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_bioes(testa_esp)
prediccions = model_BIOES_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")


Total d'entitats: 52923
Entitats predites correctament: 44792
Precisió d'entitats: 84.63616952931618 %


In [46]:
from itertools import product

# Llista de totes les combinacions possibles de valors booleans pels paràmetres
param_combinations = list(product([True, False], repeat=3))

for params in param_combinations:
    # Desempaquetem els valors booleans pels paràmetres
    use_basic_features, use_prefix_suffix_features, use_context_features = params
    
    print(f"Prova amb use_basic_features={use_basic_features}, "
          f"use_prefix_suffix_features={use_prefix_suffix_features}, "
          f"use_context_features={use_context_features}:")
    
    # Creem l'extractor de característiques amb els paràmetres corresponents
    feature_extractor = FeatureExtractor2(use_basic_features=use_basic_features, 
                                           use_prefix_suffix_features=use_prefix_suffix_features, 
                                           use_context_features=use_context_features, 
                                           pattern=r'\d+')

    train_esp_pre_BIOES = convert_to_bioes(train_esp)

    # Entrenem el model CRFTagger amb l'esquema IO
    model_BIOES_esp = CRFTagger(feature_func=feature_extractor._get_features)
    model_BIOES_esp.train(train_esp_pre_IO, 'model_io.crf.tagger')

    # Inicialitzem els comptadors
    total_entitats = 0
    entitats_correctes = 0

    # Convertim les dades de prova a format IO
    dades_prova = convert_to_bioes(testa_esp)
    prediccions = model_BIOES_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

    # Iterem sobre cada mostra en el conjunt de dades de prova
    for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
        for paraula_certes, etiqueta_certes in frase_certes:
            for paraula_pred, etiqueta_pred in frase_prediccions:
                if paraula_certes == paraula_pred:
                    total_entitats += 1
                    if etiqueta_certes == etiqueta_pred:
                        entitats_correctes += 1
                    break

    # Calculem el percentatge d'entitats predites correctament
    precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

    print("Total d'entitats:", total_entitats)
    print("Entitats predites correctament:", entitats_correctes)
    print("Precisió d'entitats:", precisio_entitats, "%")
    print("-" * 50)


Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=False:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=True:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=False:
Total d'entitats: 52923
Entitats predites correctament: 45072
Precisió d'entitats: 85.16524006575591 %
--------------------------------------------------


In [ ]:
feature_extractor = FeatureExtractor2(use_basic_features = False, use_prefix_suffix_features = False, use_context_features = False, pattern = r'\d+')

train_esp_pre_BIOES = convert_to_bioes(train_esp)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOES_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_esp.train(train_esp_pre_BIOW, 'model_bioes.crf.tagger')

# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_bioes(testb_esp)
prediccions = model_BIOES_esp.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

ned

In [47]:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

train_ned_pre_BIOES = convert_to_biow(train_ned)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOES_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_ned.train(train_ned_pre_BIOES, 'model_bioes.crf.tagger')

In [48]:
# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_bioes(testa_ned)
prediccions = model_BIOES_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

Total d'entitats: 37687
Entitats predites correctament: 33903
Precisió d'entitats: 89.95940244646695 %


In [49]:
from itertools import product

# Llista de totes les combinacions possibles de valors booleans pels paràmetres
param_combinations = list(product([True, False], repeat=3))

for params in param_combinations:
    # Desempaquetem els valors booleans pels paràmetres
    use_basic_features, use_prefix_suffix_features, use_context_features = params
    
    print(f"Prova amb use_basic_features={use_basic_features}, "
          f"use_prefix_suffix_features={use_prefix_suffix_features}, "
          f"use_context_features={use_context_features}:")
    
    # Creem l'extractor de característiques amb els paràmetres corresponents
    feature_extractor = FeatureExtractor2(use_basic_features=use_basic_features, 
                                           use_prefix_suffix_features=use_prefix_suffix_features, 
                                           use_context_features=use_context_features, 
                                           pattern=r'\d+')

    train_ned_pre_BIOES = convert_to_bioes(train_ned)

    # Entrenem el model CRFTagger amb l'esquema IO
    model_BIOES_ned = CRFTagger(feature_func=feature_extractor._get_features)
    model_BIOES_ned.train(train_ned_pre_IO, 'model_biow.crf.tagger')

    # Inicialitzem els comptadors
    total_entitats = 0
    entitats_correctes = 0

    # Convertim les dades de prova a format BIOES
    dades_prova = convert_to_bioes(testa_ned)
    prediccions = model_BIOES_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

    # Iterem sobre cada mostra en el conjunt de dades de prova
    for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
        for paraula_certes, etiqueta_certes in frase_certes:
            for paraula_pred, etiqueta_pred in frase_prediccions:
                if paraula_certes == paraula_pred:
                    total_entitats += 1
                    if etiqueta_certes == etiqueta_pred:
                        entitats_correctes += 1
                    break

    # Calculem el percentatge d'entitats predites correctament
    precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

    print("Total d'entitats:", total_entitats)
    print("Entitats predites correctament:", entitats_correctes)
    print("Precisió d'entitats:", precisio_entitats, "%")
    print("-" * 50)


Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=True, use_context_features=False:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=True:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------
Prova amb use_basic_features=True, use_prefix_suffix_features=False, use_context_features=False:
Total d'entitats: 37687
Entitats predites correctament: 33973
Precisió d'entitats: 90.14514288746783 %
--------------------------------------------------


In [ ]:
feature_extractor = FeatureExtractor2(use_basic_features = False, use_prefix_suffix_features = False, use_context_features = False, pattern = r'\d+')

train_ned_pre_BIOES = convert_to_biow(train_ned)

# Entrenar el modelo CRFTagger con el esquema IO
model_BIOES_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_ned.train(train_ned_pre_BIOES, 'model_bioes.crf.tagger')

# Inicialitzem els comptadors
total_entitats = 0
entitats_correctes = 0

# Convertim les dades de prova a format BIOES
dades_prova = convert_to_bioes(testb_ned)
prediccions = model_BIOES_ned.tag_sents([[paraula for paraula, _ in frase] for frase in dades_prova])

# Iterem sobre cada mostra en el conjunt de dades de prova
for frase_certes, frase_prediccions in zip(dades_prova, prediccions):
    for paraula_certes, etiqueta_certes in frase_certes:
        for paraula_pred, etiqueta_pred in frase_prediccions:
            
            if paraula_certes == paraula_pred:
                total_entitats += 1
                
                if etiqueta_certes == etiqueta_pred:
                    entitats_correctes += 1
                    
                break

# Calculem el percentatge d'entitats predites correctament
precisio_entitats = (entitats_correctes / total_entitats) * 100 if total_entitats > 0 else 0

print("Total d'entitats:", total_entitats)
print("Entitats predites correctament:", entitats_correctes)
print("Precisió d'entitats:", precisio_entitats, "%")

etc...

In [50]:
model_BIO.tag(nltk.word_tokenize("Mark Pedersen treballa a Google des del 1994."))

[('Mark', 'B-PER'),
 ('Pedersen', 'I-PER'),
 ('treballa', 'O'),
 ('a', 'B-PER'),
 ('Google', 'I-PER'),
 ('des', 'O'),
 ('del', 'O'),
 ('1994', 'O'),
 ('.', 'O')]

In [51]:
model_IO.tag(nltk.word_tokenize("Mark Pedersen treballa a Google des del 1994."))

NameError: name 'model_IO' is not defined

## Avaluació

In [ ]:
# Avaluació mal feta, contant només quants tokens són correctes, i no les entitats correctes.
model.accuracy(testa_esp_pre)

0.9459214330253387

In [ ]:
# Avaluació ben feta:


Hem d'avaluar quantes entitats estan reconegudes correctament, no quants tokens son correctes.
Descodificar la sequencia i obtenir les entitats, i doncs avaluar les entitats.
Per exemple, 'Mark Pedersen Romero' --> 'M P R' (una entitat) per BIO; 'M' i 'P R' (dos entitats) per IO; en aquest exemple IO ho fa malament.

A nivell d'entitats: Recall i f-score

Per avaluar el model avaluem en base a recall i precisio parcial.

## Exemple d'ús CRFTagger

In [ ]:
import unicodedata
import re

class FeatureExtractor:
    def __init__(self, pattern):
        self._pattern = pattern

    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags
            - longitud

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
        token = tokens[idx]

        feature_list = []

        if not token:
            return feature_list

        # Capitalization
        if token[0].isupper():
            feature_list.append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token) is not None:
            feature_list.append("HAS_NUM")

        # Punctuation
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token):
            feature_list.append("PUNCTUATION")
            
        # preffix up to length 3
        if len(token) > 1:
            feature_list.append("PRE_" + token[:1])
        if len(token) > 2:
            feature_list.append("PRE_" + token[:2])
        if len(token) > 3:
            feature_list.append("PRE_" + token[:3])

        # Suffix up to length 3
        if len(token) > 1:
            feature_list.append("SUF_" + token[-1:])
        if len(token) > 2:
            feature_list.append("SUF_" + token[-2:])
        if len(token) > 3:
            feature_list.append("SUF_" + token[-3:])
        
        # POS_tags
        POS = model_tagger.tag(tokens)
            
        # Paraules prèvies amb POS
        if idx > 0:
            feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
        if idx > 1:
            feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
            
        # Paraules posteriors amb POS
        if idx < (len(tokens)-1):
            feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
        if idx < (len(tokens)-2):
            feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

        feature_list.append("WORD_" + token)

        return feature_list

# Ejemplo de uso:
pattern = r'\d+'  # Patrón para encontrar números
feature_extractor = FeatureExtractor(pattern)

tokens = ['El', 'men', 'atendió', 'a', 'la', 'reunión']

for i, token in enumerate(tokens):
    features = feature_extractor._get_features(tokens, i)
    print(f"Token: {token}, Features: {features}")

Token: El, Features: ['CAPITALIZATION', 'PRE_E', 'SUF_l', 'posterior1_men_NC', 'posterior2_atendió_VMI', 'WORD_El']
Token: men, Features: ['PRE_m', 'PRE_me', 'SUF_n', 'SUF_en', 'anterior1_El_DA', 'posterior1_atendió_VMI', 'posterior2_a_SP', 'WORD_men']
Token: atendió, Features: ['PRE_a', 'PRE_at', 'PRE_ate', 'SUF_ó', 'SUF_ió', 'SUF_dió', 'anterior1_men_NC', 'anterior2_El_DA', 'posterior1_a_SP', 'posterior2_la_DA', 'WORD_atendió']
Token: a, Features: ['anterior1_atendió_VMI', 'anterior2_men_NC', 'posterior1_la_DA', 'posterior2_reunión_NC', 'WORD_a']
Token: la, Features: ['PRE_l', 'SUF_a', 'anterior1_a_SP', 'anterior2_atendió_VMI', 'posterior1_reunión_NC', 'WORD_la']
Token: reunión, Features: ['PRE_r', 'PRE_re', 'PRE_reu', 'SUF_n', 'SUF_ón', 'SUF_ión', 'anterior1_la_DA', 'anterior2_a_SP', 'WORD_reunión']
